In [0]:
from pyspark.sql.functions import col, split, to_date
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import round
from pyspark.sql.functions import sum


In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[*, 4] 
 AppName 
 Databricks Shell

### Mounting the data bricks with ADLS

In [0]:
dbutils.fs.mounts()

Out[2]: [MountInfo(mountPoint='/databricks-datasets', source='databricks-datasets', encryptionType=''),
 MountInfo(mountPoint='/Volumes', source='UnityCatalogVolumes', encryptionType=''),
 MountInfo(mountPoint='/mnt/mount1', source='wasbs://newcont@adlsraj2.blob.core.windows.net', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-tracking', source='databricks/mlflow-tracking', encryptionType=''),
 MountInfo(mountPoint='/mnt/mount_point_case_study', source='wasbs://wallmart@adlsraj2.blob.core.windows.net', encryptionType=''),
 MountInfo(mountPoint='/mnt/mount_point_sas', source='wasbs://newcont@adlsraj2.blob.core.windows.net', encryptionType=''),
 MountInfo(mountPoint='/databricks-results', source='databricks-results', encryptionType=''),
 MountInfo(mountPoint='/mnt/mount_point1', source='wasbs://newcont@adlsraj2.blob.core.windows.net', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-registry', source='databricks/mlflow-registry', encryptionType=''),
 MountInfo

In [0]:

# try :

#  dbutils.fs.mount(
#     source= 'wasbs://test@adlsacid.blob.core.windows.net',
#     #//container_name@account_name
#     mount_point ='/mnt/projectnew_mount/',
#     extra_configs= {
#         #dictonary keywill be url, value will be key of the storage account in azure 
#           "fs.azure.account.key.adlsacid.blob.core.windows.net":"PLQ88zVYwXYLD+EKLei7rvyMRAtprs9Y6Z7TeIJ1Q0bNkr4YmzN0pCmUh1Vxwd35dTW4WKPZ/gHY+ASt3Cypjw=="
#     }
# )
# else :
#     dbutils.fs.unmount('/mnt/projectnew_mount')


Out[111]: True

In [0]:
try:
    # Check if the mount point is already mounted
    existing_mount = next((mount for mount in dbutils.fs.mounts() if mount.mountPoint == '/mnt/projectnew_mount'), None)
    
    if existing_mount is None:
        # Mount the storage if it's not already mounted
        dbutils.fs.mount(
            source='wasbs://test@adlsacid.blob.core.windows.net',
            mount_point='/mnt/projectnew_mount/',
            extra_configs={
                "fs.azure.account.key.adlsacid.blob.core.windows.net": "PLQ88zVYwXYLD+EKLei7rvyMRAtprs9Y6Z7TeIJ1Q0bNkr4YmzN0pCmUh1Vxwd35dTW4WKPZ/gHY+ASt3Cypjw=="
            }
        )
    else:
        print("Mount already exists:", existing_mount.source)
except Exception as e:
    print("Error:", str(e))

# Continue with the rest of your notebook code here


Error: An error occurred while calling o1711.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/projectnew_mount; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/projectnew_mount
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:135)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:69)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.createOrUpdateMount(DBUtilsCore.scala:1025)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.$anonfun$mount$1(DBUtilsCore.scala:1051)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:555)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:650)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:671)
	at com.databricks.l

In [0]:
dbutils.fs.ls('/mnt/projectnew_mount/')

Out[112]: [FileInfo(path='dbfs:/mnt/projectnew_mount/amazon prime movies.csv', name='amazon prime movies.csv', size=2439189, modificationTime=1691643181000),
 FileInfo(path='dbfs:/mnt/projectnew_mount/cleaned/', name='cleaned/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/projectnew_mount/disney_plus_shows.csv', name='disney_plus_shows.csv', size=406103, modificationTime=1691637652000),
 FileInfo(path='dbfs:/mnt/projectnew_mount/netflix_titles.csv', name='netflix_titles.csv', size=3315606, modificationTime=1691637652000),
 FileInfo(path='dbfs:/mnt/projectnew_mount/results/', name='results/', size=0, modificationTime=0)]

In [0]:
df_amazon=spark.read.csv('/mnt/projectnew_mount/amazon prime movies.csv', inferSchema=True, header=True)
df_amazon.show(4)

+------------+--------+------+----------+-------------+----------+--------------------+
| movie_title|language|rating|   runtime|year_released|age_rating|              review|
+------------+--------+------+----------+-------------+----------+--------------------+
|  John Rambo|   Tamil|     7|1 h 26 min|         2008|       18+|In Thailand John ...|
|American Pie| English|     7|1 h 35 min|         1999|       18+|"You'll never loo...|
|   Bombshell| English|   6.8|1 h 48 min|         2019|       18+|The provocative r...|
|  Love Birds|   Tamil|   5.1|2 h 40 min|         1996|       All|After accidentall...|
+------------+--------+------+----------+-------------+----------+--------------------+
only showing top 4 rows



In [0]:
df_amazon.printSchema()

root
 |-- movie_title: string (nullable = true)
 |-- language: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- year_released: string (nullable = true)
 |-- age_rating: string (nullable = true)
 |-- review: string (nullable = true)



In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import *
df_amazon = df_amazon.withColumn('RATING', col('RATING').cast(FloatType()))
df_amazon=df_amazon.withColumn('YEAR_RELEASED', col('YEAR_RELEASED').cast(IntegerType()))


In [0]:
df_amazon.select('runtime').collect()[1][0]

Out[117]: '1 h 35 min'

In [0]:
split_col = split(df_amazon["runtime"], " ")

In [0]:
df_amazon = df_amazon.withColumn("HOURS", split_col[0].cast("int"))
df_amazon = df_amazon.withColumn("MINUTES", split_col[2].cast("int"))

In [0]:
from pyspark.sql.functions import round
df_amazon = df_amazon.withColumn("DURATION_NEW",
                                 round(col("HOURS") + col("MINUTES") / 60, 3)
                                 )

In [0]:
df_amazon=df_amazon.drop('HOURS','MINUTES')
df_amazon.show(4)

+------------+--------+------+----------+-------------+----------+--------------------+------------+
| movie_title|language|RATING|   runtime|YEAR_RELEASED|age_rating|              review|DURATION_NEW|
+------------+--------+------+----------+-------------+----------+--------------------+------------+
|  John Rambo|   Tamil|   7.0|1 h 26 min|         2008|       18+|In Thailand John ...|       1.433|
|American Pie| English|   7.0|1 h 35 min|         1999|       18+|"You'll never loo...|       1.583|
|   Bombshell| English|   6.8|1 h 48 min|         2019|       18+|The provocative r...|         1.8|
|  Love Birds|   Tamil|   5.1|2 h 40 min|         1996|       All|After accidentall...|       2.667|
+------------+--------+------+----------+-------------+----------+--------------------+------------+
only showing top 4 rows



In [0]:
df_amazon.printSchema()

root
 |-- movie_title: string (nullable = true)
 |-- language: string (nullable = true)
 |-- RATING: float (nullable = true)
 |-- runtime: string (nullable = true)
 |-- YEAR_RELEASED: integer (nullable = true)
 |-- age_rating: string (nullable = true)
 |-- review: string (nullable = true)
 |-- DURATION_NEW: double (nullable = true)



In [0]:
is_null_column = df_amazon.select(col("movie_title").isNull().alias("is_null")).count()
is_null_column

Out[123]: 8129

In [0]:
from pyspark.sql.functions import sum
null_value_counts = df_amazon.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_amazon.columns])
null_value_counts.show()

+-----------+--------+------+-------+-------------+----------+------+------------+
|movie_title|language|RATING|runtime|YEAR_RELEASED|age_rating|review|DURATION_NEW|
+-----------+--------+------+-------+-------------+----------+------+------------+
|          0|       2|  2010|      2|          399|         2|     7|         649|
+-----------+--------+------+-------+-------------+----------+------+------------+



In [0]:
df_amazon.write.mode('overwrite').csv('/mnt/projectnew_mount/cleaned/amazon',header=True)

## Transformation of Disney

In [0]:
#read the csv file after mounting the databricks to adls
custom_delimiter = r","
df_disney=spark.read.option("delimiter", custom_delimiter).csv('/mnt/projectnew_mount/disney_plus_shows.csv', inferSchema=True, header=True)

In [0]:
#see the sample data
display(df_disney.limit(10))

imdb_id,title,plot,type,rated,year,released_at,added_at,runtime,genre,director,writer,actors,language,country,awards,metascore,imdb_rating,imdb_votes
tt0147800,10 Things I Hate About You,A pretty popular teenager can't go out on a date until her ill-tempered older sister does.,movie,PG-13,1999,31-Mar-99,12-Nov-19,97 min,Comedy\t Drama\t Romance,Gil Junger,Karen McCullah\t Kirsten Smith,Heath Ledger\t Julia Stiles\t Joseph Gordon-Levitt\t Larisa Oleynik,English\t French,USA,2 wins & 13 nominations.,70,7.3,283945
tt7019028,101 Dalmatian Street,This series follows the lives of Delilah and Doug descendants of Pongo and Perdita and their 99 dalmatian pups at 101 Dalmatian Street Camden Town.,series,N/A,2018–,25-Mar-19,28-Feb-20,N/A,Animation\t Comedy\t Family,N/A,N/A,Josh Brener\t Michaela Dietz\t Bert Davis\t Abigail Zoe Lewis,English,UK\t USA\t Canada,N/A,N/A,6.2,124
tt0115433,101 Dalmatians,An evil high-fashion designer plots to steal Dalmatian puppies in order to make an extravagant fur coat but instead creates an extravagant mess.,movie,G,1996,27-Nov-96,12-Nov-19,103 min,Adventure\t Comedy\t Crime\t Family,Stephen Herek,Dodie Smith (novel)\t John Hughes (screenplay),Glenn Close\t Jeff Daniels\t Joely Richardson\t Joan Plowright,English\t Spanish,USA\t UK,Nominated for 1 Golden Globe. Another 3 wins & 9 nominations.,49,5.7,97785
tt0324941,101 Dalmatians 2: Patch's London Adventure,Being one of 101 takes its toll on Patch who doesn't feel unique. When he's accidentally left behind on moving day he meets his idol Thunderbolt who enlists him on a publicity campaign.,movie,G,2002,21-Jan-03,12-Nov-19,74 min,Animation\t Adventure\t Comedy\t Family\t Musical,Jim Kammerud\t Brian Smith,Jim Kammerud (story)\t Dan Root (story)\t Garrett K. Schiff (story)\t Brian Smith (story)\t Jim Kammerud (screenplay)\t Brian Smith (screenplay)\t Dodie Smith (novel)\t Michael Lucker\t Temple Mathews (additional script material),Barry Bostwick\t Jason Alexander\t Martin Short\t Bobby Lockwood,English,USA,5 wins & 10 nominations.,N/A,5.8,7434
tt0211181,102 Dalmatians,Cruella DeVil gets out of prison and goes after the puppies once more.,movie,G,2000,22-Nov-00,12-Nov-19,100 min,Adventure\t Comedy\t Family,Kevin Lima,Dodie Smith (novel)\t Kristen Buckley (story)\t Brian Regan (story)\t Kristen Buckley (screenplay)\t Brian Regan (screenplay)\t Bob Tzudiker (screenplay)\t Noni White (screenplay),Glenn Close\t Gérard Depardieu\t Ioan Gruffudd\t Alice Evans,English,USA\t UK,Nominated for 1 Oscar. Another 1 win & 7 nominations.,35,4.9,33444
tt1846442,12 Dates of Christmas,A story that follows Kate a young woman who after a horrible blind date on Christmas Eve wakes up to find she is re-living that same day and date all over again.,movie,PG,2011,11-Dec-11,12-Nov-19,90 min,Comedy\t Fantasy\t Romance,James Hayman,Aaron Mendelsohn (teleplay)\t Janet Brownell (teleplay)\t Aaron Mendelsohn (story)\t Blake Harris (story),Laura Miyata\t Vijay Mehta\t Amy Smart\t Audrey Dwyer,English,USA,1 nomination.,N/A,6.3,4933
tt0046672,20000 Leagues Under the Sea,A ship sent to investigate a wave of mysterious sinkings encounters the advanced submarine the Nautilus commanded by Captain Nemo.,movie,G,1954,20-Jul-55,12-Nov-19,127 min,Adventure\t Drama\t Family\t Fantasy\t Sci-Fi,Richard Fleischer,Earl Felton (screenplay),Kirk Douglas\t James Mason\t Paul Lukas\t Peter Lorre,English,USA,Won 2 Oscars. Another 3 wins & 3 nominations.,83,7.2,27761
tt0120623,A Bug's Life,"""A misfit ant looking for """"warriors"""" to save his colony from greedy grasshoppers recruits a group of bugs that turn out to be an inept circus troupe.""",movie,G,1998,25-Nov-98,12-Nov-19,95 min,Animation\t Adventure\t Comedy\t Family,John Lasseter\t Andrew Stanton(co-director),John Lasseter (original story by)\t Andrew Stanton (original story by)\t Joe Ranft (original story by)\t Andrew Stanton (screenplay by)\t Don McEnery (screenplay by)\t Bob Shaw (screenplay by),Dave Foley\t Kevin Spacey\t Julia Louis-Dreyfus\t

In [0]:
df_disney.printSchema()

root
 |-- imdb_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- plot: string (nullable = true)
 |-- type: string (nullable = true)
 |-- rated: string (nullable = true)
 |-- year: string (nullable = true)
 |-- released_at: string (nullable = true)
 |-- added_at: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- director: string (nullable = true)
 |-- writer: string (nullable = true)
 |-- actors: string (nullable = true)
 |-- language: string (nullable = true)
 |-- country: string (nullable = true)
 |-- awards: string (nullable = true)
 |-- metascore: string (nullable = true)
 |-- imdb_rating: string (nullable = true)
 |-- imdb_votes: string (nullable = true)



In [0]:
#casting the year column datatype to integer
df_disney = df_disney.withColumn('year', col('year').cast(IntegerType()))

In [0]:
# chnagingsome other columns data types
df_disney = df_disney.withColumn('metascore', col('metascore').cast(IntegerType()))
df_disney = df_disney.withColumn('imdb_rating', col('imdb_rating').cast(FloatType()))


In [0]:
#changing one value format to min
from pyspark.sql.functions import when
df_disney = df_disney.withColumn("runtime",
                                 when(col("runtime") == '1 h', '60 min')
                                 .otherwise(col("runtime")))

In [0]:

# changing the format of the run time time to integer
from pyspark.sql.functions import split
df_disney = df_disney.withColumn("runtime", split(col("runtime"), " ")[0].cast("int"))


In [0]:
#see the schema once
df_disney.printSchema()

root
 |-- imdb_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- plot: string (nullable = true)
 |-- type: string (nullable = true)
 |-- rated: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- released_at: string (nullable = true)
 |-- added_at: string (nullable = true)
 |-- runtime: integer (nullable = true)
 |-- genre: string (nullable = true)
 |-- director: string (nullable = true)
 |-- writer: string (nullable = true)
 |-- actors: string (nullable = true)
 |-- language: string (nullable = true)
 |-- country: string (nullable = true)
 |-- awards: string (nullable = true)
 |-- metascore: integer (nullable = true)
 |-- imdb_rating: float (nullable = true)
 |-- imdb_votes: string (nullable = true)



In [0]:
# changing the date format from d-MMM-yy to yyyy-MM-dd for that we need pivot point for spark to understand
pivot_year = 1930
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
spark.conf.set("spark.sql.legacy.timeParserPolicy.pivotYear", pivot_year)

In [0]:
# changing the format now 
from pyspark.sql.functions import to_date
df_disney = df_disney.withColumn("released_date", to_date(col("released_at"), "d-MMM-yy"))


In [0]:
#changing the format of added_at column date format
from pyspark.sql.functions import date_format
df_disney = df_disney.withColumn("added_date", to_date(col("added_at"), "d-MMM-yy"))
# November 12, 2019

In [0]:
#check the data once

display(df_disney.limit(10))

imdb_id,title,plot,type,rated,year,released_at,added_at,runtime,genre,director,writer,actors,language,country,awards,metascore,imdb_rating,imdb_votes,released_date,added_date
tt0147800,10 Things I Hate About You,A pretty popular teenager can't go out on a date until her ill-tempered older sister does.,movie,PG-13,1999,31-Mar-99,12-Nov-19,97,Comedy\t Drama\t Romance,Gil Junger,Karen McCullah\t Kirsten Smith,Heath Ledger\t Julia Stiles\t Joseph Gordon-Levitt\t Larisa Oleynik,English\t French,USA,2 wins & 13 nominations.,70,7.3,283945,1999-03-31,2019-11-12
tt7019028,101 Dalmatian Street,This series follows the lives of Delilah and Doug descendants of Pongo and Perdita and their 99 dalmatian pups at 101 Dalmatian Street Camden Town.,series,N/A,null,25-Mar-19,28-Feb-20,null,Animation\t Comedy\t Family,N/A,N/A,Josh Brener\t Michaela Dietz\t Bert Davis\t Abigail Zoe Lewis,English,UK\t USA\t Canada,N/A,null,6.2,124,2019-03-25,2020-02-28
tt0115433,101 Dalmatians,An evil high-fashion designer plots to steal Dalmatian puppies in order to make an extravagant fur coat but instead creates an extravagant mess.,movie,G,1996,27-Nov-96,12-Nov-19,103,Adventure\t Comedy\t Crime\t Family,Stephen Herek,Dodie Smith (novel)\t John Hughes (screenplay),Glenn Close\t Jeff Daniels\t Joely Richardson\t Joan Plowright,English\t Spanish,USA\t UK,Nominated for 1 Golden Globe. Another 3 wins & 9 nominations.,49,5.7,97785,1996-11-27,2019-11-12
tt0324941,101 Dalmatians 2: Patch's London Adventure,Being one of 101 takes its toll on Patch who doesn't feel unique. When he's accidentally left behind on moving day he meets his idol Thunderbolt who enlists him on a publicity campaign.,movie,G,2002,21-Jan-03,12-Nov-19,74,Animation\t Adventure\t Comedy\t Family\t Musical,Jim Kammerud\t Brian Smith,Jim Kammerud (story)\t Dan Root (story)\t Garrett K. Schiff (story)\t Brian Smith (story)\t Jim Kammerud (screenplay)\t Brian Smith (screenplay)\t Dodie Smith (novel)\t Michael Lucker\t Temple Mathews (additional script material),Barry Bostwick\t Jason Alexander\t Martin Short\t Bobby Lockwood,English,USA,5 wins & 10 nominations.,null,5.8,7434,2003-01-21,2019-11-12
tt0211181,102 Dalmatians,Cruella DeVil gets out of prison and goes after the puppies once more.,movie,G,2000,22-Nov-00,12-Nov-19,100,Adventure\t Comedy\t Family,Kevin Lima,Dodie Smith (novel)\t Kristen Buckley (story)\t Brian Regan (story)\t Kristen Buckley (screenplay)\t Brian Regan (screenplay)\t Bob Tzudiker (screenplay)\t Noni White (screenplay),Glenn Close\t Gérard Depardieu\t Ioan Gruffudd\t Alice Evans,English,USA\t UK,Nominated for 1 Oscar. Another 1 win & 7 nominations.,35,4.9,33444,2000-11-22,2019-11-12
tt1846442,12 Dates of Christmas,A story that follows Kate a young woman who after a horrible blind date on Christmas Eve wakes up to find she is re-living that same day and date all over again.,movie,PG,2011,11-Dec-11,12-Nov-19,90,Comedy\t Fantasy\t Romance,James Hayman,Aaron Mendelsohn (teleplay)\t Janet Brownell (teleplay)\t Aaron Mendelsohn (story)\t Blake Harris (story),Laura Miyata\t Vijay Mehta\t Amy Smart\t Audrey Dwyer,English,USA,1 nomination.,null,6.3,4933,2011-12-11,2019-11-12
tt0046672,20000 Leagues Under the Sea,A ship sent to investigate a wave of mysterious sinkings encounters the advanced submarine the Nautilus commanded by Captain Nemo.,movie,G,1954,20-Jul-55,12-Nov-19,127,Adventure\t Drama\t Family\t Fantasy\t Sci-Fi,Richard Fleischer,Earl Felton (screenplay),Kirk Douglas\t James Mason\t Paul Lukas\t Peter Lorre,English,USA,Won 2 Oscars. Another 3 wins & 3 nominations.,83,7.2,27761,1955-07-20,2019-11-12
tt0120623,A Bug's Life,"""A misfit ant looking for """"warriors"""" to save his colony from greedy grasshoppers recruits a group of bugs that turn out to be an inept circus troupe.""",movie,G,1998,25-Nov-98,12-Nov-19,95,Animation\t Adventure\t Comedy\t Family,John Lasseter\t Andrew Stanton(co-director),John Lasseter (original story by)\t Andrew Stanton (original story by)\t Joe Ranft (orig

In [0]:
display(df_disney.filter(df_disney.title =='Frozen'))

imdb_id,title,plot,type,rated,year,released_at,added_at,runtime,genre,director,writer,actors,language,country,awards,metascore,imdb_rating,imdb_votes,released_date,added_date
tt2294629,Frozen,When the newly crowned Queen Elsa accidentally uses her power to turn things into ice to curse her home in infinite winter her sister Anna teams up with a mountain man his playful reindeer and a snowman to change the weather condition.,movie,PG,2013,27-Nov-13,12-Nov-19,102,Animation\t Adventure\t Comedy\t Family\t Fantasy\t Musical,Chris Buck\t Jennifer Lee,"""Jennifer Lee (screenplay by)\t Hans Christian Andersen (story inspired by: """"The Snow Queen"""" by)\t Chris Buck (story by)\t Jennifer Lee (story by)\t Shane Morris (story by)""",Kristen Bell\t Idina Menzel\t Jonathan Groff\t Josh Gad,English\t Norwegian,USA,Won 2 Oscars. Another 79 wins & 59 nominations.,75,7.5,551637,2013-11-27,2019-11-12


In [0]:
from pyspark.sql.functions import regexp_extract
wins_pattern = r"(\d+)\s+(?:win|wins)"
nominations_pattern = r"(\d+)\s+(?:nomination|nominations)"

# Use regexp_extract to extract numeric values for wins and nominations
df_disney = df_disney.withColumn("Wins", regexp_extract(col("awards"), wins_pattern, 1).cast("int"))
df_disney = df_disney.withColumn("Nominations", regexp_extract(col("awards"), nominations_pattern, 1).cast("int"))

# # Show the resulting DataFrame
# disney.show(truncate=False)

# Show the resulting DataFrame
df_disney.select(col('Wins')).show(truncate=False)

# Comparing nulls
df_disney.select(sum(col('Wins').isNull().cast("int")).alias('Winsnull')).show()
df_disney.filter(col("Wins").isNull() & col("Nominations").isNull()).count()
df_disney.select(sum(col('Nominations').isNull().cast("int")).alias('Nominationsnull')).show()

+----+
|Wins|
+----+
|2   |
|null|
|3   |
|5   |
|1   |
|null|
|3   |
|14  |
|null|
|null|
|null|
|null|
|2   |
|null|
|5   |
|7   |
|31  |
|2   |
|null|
|null|
+----+
only showing top 20 rows

+--------+
|Winsnull|
+--------+
|     616|
+--------+

+---------------+
|Nominationsnull|
+---------------+
|            481|
+---------------+



In [0]:
df_disney.rdd.getNumPartitions()

Out[87]: 1

In [0]:
#drop unnecessary columns
df_disney=df_disney.drop('released_at', 'added_at')

In [0]:
df_disney.columns

Out[89]: ['imdb_id',
 'title',
 'plot',
 'type',
 'rated',
 'year',
 'runtime',
 'genre',
 'director',
 'writer',
 'actors',
 'language',
 'country',
 'awards',
 'metascore',
 'imdb_rating',
 'imdb_votes',
 'released_date',
 'added_date',
 'Wins',
 'Nominations']

In [0]:
from pyspark.sql.functions import col, regexp_replace
# Convert 'value' column to integer
df_disney = df_disney.withColumn("imdb_votes_for", col("imdb_votes").cast("int"))

In [0]:
display(df_disney.limit(10))

imdb_id,title,plot,type,rated,year,runtime,genre,director,writer,actors,language,country,awards,metascore,imdb_rating,imdb_votes,released_date,added_date,Wins,Nominations,imdb_votes_for
tt0147800,10 Things I Hate About You,A pretty popular teenager can't go out on a date until her ill-tempered older sister does.,movie,PG-13,1999,97,Comedy\t Drama\t Romance,Gil Junger,Karen McCullah\t Kirsten Smith,Heath Ledger\t Julia Stiles\t Joseph Gordon-Levitt\t Larisa Oleynik,English\t French,USA,2 wins & 13 nominations.,70,7.3,283945,1999-03-31,2019-11-12,2,13,283945
tt7019028,101 Dalmatian Street,This series follows the lives of Delilah and Doug descendants of Pongo and Perdita and their 99 dalmatian pups at 101 Dalmatian Street Camden Town.,series,N/A,null,null,Animation\t Comedy\t Family,N/A,N/A,Josh Brener\t Michaela Dietz\t Bert Davis\t Abigail Zoe Lewis,English,UK\t USA\t Canada,N/A,null,6.2,124,2019-03-25,2020-02-28,null,null,124
tt0115433,101 Dalmatians,An evil high-fashion designer plots to steal Dalmatian puppies in order to make an extravagant fur coat but instead creates an extravagant mess.,movie,G,1996,103,Adventure\t Comedy\t Crime\t Family,Stephen Herek,Dodie Smith (novel)\t John Hughes (screenplay),Glenn Close\t Jeff Daniels\t Joely Richardson\t Joan Plowright,English\t Spanish,USA\t UK,Nominated for 1 Golden Globe. Another 3 wins & 9 nominations.,49,5.7,97785,1996-11-27,2019-11-12,3,9,97785
tt0324941,101 Dalmatians 2: Patch's London Adventure,Being one of 101 takes its toll on Patch who doesn't feel unique. When he's accidentally left behind on moving day he meets his idol Thunderbolt who enlists him on a publicity campaign.,movie,G,2002,74,Animation\t Adventure\t Comedy\t Family\t Musical,Jim Kammerud\t Brian Smith,Jim Kammerud (story)\t Dan Root (story)\t Garrett K. Schiff (story)\t Brian Smith (story)\t Jim Kammerud (screenplay)\t Brian Smith (screenplay)\t Dodie Smith (novel)\t Michael Lucker\t Temple Mathews (additional script material),Barry Bostwick\t Jason Alexander\t Martin Short\t Bobby Lockwood,English,USA,5 wins & 10 nominations.,null,5.8,7434,2003-01-21,2019-11-12,5,10,7434
tt0211181,102 Dalmatians,Cruella DeVil gets out of prison and goes after the puppies once more.,movie,G,2000,100,Adventure\t Comedy\t Family,Kevin Lima,Dodie Smith (novel)\t Kristen Buckley (story)\t Brian Regan (story)\t Kristen Buckley (screenplay)\t Brian Regan (screenplay)\t Bob Tzudiker (screenplay)\t Noni White (screenplay),Glenn Close\t Gérard Depardieu\t Ioan Gruffudd\t Alice Evans,English,USA\t UK,Nominated for 1 Oscar. Another 1 win & 7 nominations.,35,4.9,33444,2000-11-22,2019-11-12,1,7,33444
tt1846442,12 Dates of Christmas,A story that follows Kate a young woman who after a horrible blind date on Christmas Eve wakes up to find she is re-living that same day and date all over again.,movie,PG,2011,90,Comedy\t Fantasy\t Romance,James Hayman,Aaron Mendelsohn (teleplay)\t Janet Brownell (teleplay)\t Aaron Mendelsohn (story)\t Blake Harris (story),Laura Miyata\t Vijay Mehta\t Amy Smart\t Audrey Dwyer,English,USA,1 nomination.,null,6.3,4933,2011-12-11,2019-11-12,null,1,4933
tt0046672,20000 Leagues Under the Sea,A ship sent to investigate a wave of mysterious sinkings encounters the advanced submarine the Nautilus commanded by Captain Nemo.,movie,G,1954,127,Adventure\t Drama\t Family\t Fantasy\t Sci-Fi,Richard Fleischer,Earl Felton (screenplay),Kirk Douglas\t James Mason\t Paul Lukas\t Peter Lorre,English,USA,Won 2 Oscars. Another 3 wins & 3 nominations.,83,7.2,27761,1955-07-20,2019-11-12,3,3,27761
tt0120623,A Bug's Life,"""A misfit ant looking for """"warriors"""" to save his colony from greedy grasshoppers recruits a group of bugs that turn out to be an inept circus troupe.""",movie,G,1998,95,Animation\t Adventure\t Comedy\t Family,John Lasseter\t Andrew Stanton(co-director),John Lasseter (original story by)\t Andrew Stanton (original story by)\t Joe Ranft (original story by)\t Andrew Stanton (screenplay by)\t Don McEnery (screenpl

In [0]:
df_disney.select(col('imdb_votes_for')).show()

+--------------+
|imdb_votes_for|
+--------------+
|        283945|
|           124|
|         97785|
|          7434|
|         33444|
|          4933|
|         27761|
|        257065|
|            35|
|         43655|
|          5479|
|           488|
|          1042|
|            59|
|         38659|
|        204537|
|        356283|
|         31603|
|         25623|
|        123377|
+--------------+
only showing top 20 rows



In [0]:
display(df_disney.filter(df_disney.title =='Frozen'))

imdb_id,title,plot,type,rated,year,runtime,genre,director,writer,actors,language,country,awards,metascore,imdb_rating,imdb_votes,released_date,added_date,Wins,Nominations,imdb_votes_for
tt2294629,Frozen,When the newly crowned Queen Elsa accidentally uses her power to turn things into ice to curse her home in infinite winter her sister Anna teams up with a mountain man his playful reindeer and a snowman to change the weather condition.,movie,PG,2013,102,Animation\t Adventure\t Comedy\t Family\t Fantasy\t Musical,Chris Buck\t Jennifer Lee,"""Jennifer Lee (screenplay by)\t Hans Christian Andersen (story inspired by: """"The Snow Queen"""" by)\t Chris Buck (story by)\t Jennifer Lee (story by)\t Shane Morris (story by)""",Kristen Bell\t Idina Menzel\t Jonathan Groff\t Josh Gad,English\t Norwegian,USA,Won 2 Oscars. Another 79 wins & 59 nominations.,75,7.5,551637,2013-11-27,2019-11-12,79,59,551637


In [0]:
#save the cleaned data to the cleaned folder
df_disney.write.mode('overwrite').csv('/mnt/projectnew_mount/cleaned/disney/', header=True)

### Netflix dataset Transformation

In [0]:
df_netflix=spark.read.csv('/mnt/projectnew_mount/netflix_titles.csv', inferSchema=True, header=True)

In [0]:
display(df_netflix.limit(10))

show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,null,United States,25-Sep-21,2020,PG-13,90 min,Documentaries,As her father nears the end of his life filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable.
s2,TV Show,Blood & Water,null,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thabang Molaba, Dillon Windvogel, Natasha Thahane, Arno Greeff, Xolile Tshabalala, Getmore Sithole, Cindy Mahlangu, Ryle De Morny, Greteli Fincham, Sello Maake Ka-Ncube, Odwa Gwanya, Mekaila Mathys, Sandi Schultz, Duane Williams, Shamilla Miller, Patrick Mofokeng",South Africa,24-Sep-21,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries",After crossing paths at a party a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth.
s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabiha Akkari, Sofia Lesaffre, Salim Kechiouche, Noureddine Farihi, Geert Van Rampelberg, Bakary Diombera",null,24-Sep-21,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Action & Adventure",To protect his family from a powerful drug lord skilled thief Mehdi and his expert team of robbers are pulled into a violent and deadly turf war.
s4,TV Show,Jailbirds New Orleans,null,null,null,24-Sep-21,2021,TV-MA,1 Season,"Docuseries, Reality TV",Feuds flirtations and toilet talk go down among the incarcerated women at the Orleans Justice Center in New Orleans on this gritty reality series.
s5,TV Show,Kota Factory,null,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam Khan, Ahsaas Channa, Revathi Pillai, Urvi Singh, Arun Kumar",India,24-Sep-21,2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV Comedies",In a city of coaching centers known to train India’s finest collegiate minds an earnest but unexceptional student and his friends navigate campus life.
s6,TV Show,Midnight Mass,Mike Flanagan,"Kate Siegel, Zach Gilford, Hamish Linklater, Henry Thomas, Kristin Lehman, Samantha Sloyan, Igby Rigney, Rahul Kohli, Annarah Cymone, Annabeth Gish, Alex Essoe, Rahul Abburi, Matt Biedel, Michael Trucco, Crystal Balint, Louis Oliver",null,24-Sep-21,2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries",The arrival of a charismatic young priest brings glorious miracles ominous mysteries and renewed religious fervor to a dying town desperate to believe.
s7,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, Sofia Carson, Liza Koshy, Ken Jeong, Elizabeth Perkins, Jane Krakowski, Michael McKean, Phil LaMarr",null,24-Sep-21,2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero believes Earth Ponies Pegasi and Unicorns should be pals — and hoof to heart she’s determined to prove it.
s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra Duah, Nick Medley, Mutabaruka, Afemo Omilami, Reggie Carter, Mzuri","United States, Ghana, Burkina Faso, United Kingdom, Germany, Ethiopia",24-Sep-21,1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies",On a photo shoot in Ghana an American model slips back in time becomes enslaved on a plantation and bears witness to the agony of her ancestral past.
s9,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Hollywood",United Kingdom,24-Sep-21,2021,TV-14,9 Seasons,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in a 10-week competition whipping up their best dishes in the hopes of being named the U.K.'s best.
s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, Timothy Olyphant, Daveed Diggs, Skyler Gisondo, Laura Harrier, Rosalind Chao, Kimberly Quinn, Loretta Devine, Ravi Kapoor",United States,24-Sep-21,2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to l

In [0]:
df_netflix.printSchema()

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)



In [0]:
#Define the date format
date_format = "dd-MMM-yy"

# Cast the string column to a date column
df_netflix = df_netflix.withColumn("date_added", to_date(col("date_added"), date_format))

# Show the updated DataFrame
df_netflix.show()

+-------+-------+--------------------+--------------------+--------------------+--------------------+----------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|            director|                cast|             country|date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+--------------------+--------------------+--------------------+----------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|     Kirsten Johnson|                null|       United States|2021-09-25|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|                null|Ama Qamata, Khosi...|        South Africa|2021-09-24|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglands|     Julien Leclercq|Sami Bouajila, T

In [0]:
# Use regular expressions to extract numeric values
#extracted_duration = regexp_extract(col("duration"), r"(\d+)", 1)

# Convert the extracted values to integers
#df_netflix = df_netflix.withColumn("duration", extracted_duration.cast("int"))

split_values = split(col("duration"), " ")
numeric_value = split_values.getItem(0)

# Cast the extracted value to integer
df_netflix = df_netflix.withColumn("duration", numeric_value.cast("int"))
# Show the updated DataFrame
df_netflix.show(truncate=True)


+-------+-------+--------------------+--------------------+--------------------+--------------------+----------+------------+------+--------+--------------------+--------------------+
|show_id|   type|               title|            director|                cast|             country|date_added|release_year|rating|duration|           listed_in|         description|
+-------+-------+--------------------+--------------------+--------------------+--------------------+----------+------------+------+--------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|     Kirsten Johnson|                null|       United States|2021-09-25|        2020| PG-13|      90|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|                null|Ama Qamata, Khosi...|        South Africa|2021-09-24|        2021| TV-MA|       2|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglands|     Julien Leclercq|Sami Bouajila, Tr...|

In [0]:
df_netflix.select("date_added", "duration").show()

+----------+--------+
|date_added|duration|
+----------+--------+
|2021-09-25|      90|
|2021-09-24|       2|
|2021-09-24|       1|
|2021-09-24|       1|
|2021-09-24|       2|
|2021-09-24|       1|
|2021-09-24|      91|
|2021-09-24|     125|
|2021-09-24|       9|
|2021-09-24|     104|
|2021-09-24|       1|
|2021-09-23|       1|
|2021-09-23|     127|
|2021-09-22|      91|
|2021-09-22|       1|
|2021-09-22|       4|
|2021-09-22|      67|
|2021-09-22|       2|
|2021-09-22|      94|
|2021-09-22|       1|
+----------+--------+
only showing top 20 rows



In [0]:
df_netflix = df_netflix.withColumn("release_year", col("release_year").cast("int"))

In [0]:
df_netflix.select("release_year").show()

+------------+
|release_year|
+------------+
|        2020|
|        2021|
|        2021|
|        2021|
|        2021|
|        2021|
|        2021|
|        1993|
|        2021|
|        2021|
|        2021|
|        2021|
|        2021|
|        2021|
|        2021|
|        2021|
|        2020|
|        2020|
|        2021|
|        2021|
+------------+
only showing top 20 rows



In [0]:

from pyspark.sql.functions import explode
df_exploded = df_netflix.withColumn("listed_in_individual", explode(split(df_netflix.listed_in, ", ")))
df_exploded.select("listed_in","listed_in_individual").show(truncate=False)

+-------------------------------------------------------------+------------------------+
|listed_in                                                    |listed_in_individual    |
+-------------------------------------------------------------+------------------------+
|Documentaries                                                |Documentaries           |
|International TV Shows, TV Dramas, TV Mysteries              |International TV Shows  |
|International TV Shows, TV Dramas, TV Mysteries              |TV Dramas               |
|International TV Shows, TV Dramas, TV Mysteries              |TV Mysteries            |
|Crime TV Shows, International TV Shows, TV Action & Adventure|Crime TV Shows          |
|Crime TV Shows, International TV Shows, TV Action & Adventure|International TV Shows  |
|Crime TV Shows, International TV Shows, TV Action & Adventure|TV Action & Adventure   |
|Docuseries, Reality TV                                       |Docuseries              |
|Docuseries, Reality 

In [0]:
df_tv_shows = df_exploded.withColumn("country_individual", explode(split(df_exploded.country, ", ")))

In [0]:
df_tv_shows.select("country_individual").show()

+------------------+
|country_individual|
+------------------+
|     United States|
|      South Africa|
|      South Africa|
|      South Africa|
|             India|
|             India|
|             India|
|     United States|
|             Ghana|
|      Burkina Faso|
|    United Kingdom|
|           Germany|
|          Ethiopia|
|     United States|
|             Ghana|
|      Burkina Faso|
|    United Kingdom|
|           Germany|
|          Ethiopia|
|     United States|
+------------------+
only showing top 20 rows



In [0]:
df_tv_shows.columns

Out[107]: ['show_id',
 'type',
 'title',
 'director',
 'cast',
 'country',
 'date_added',
 'release_year',
 'rating',
 'duration',
 'listed_in',
 'description',
 'listed_in_individual',
 'country_individual']

In [0]:
df_tv_shows.rdd.getNumPartitions()

Out[108]: 1

In [0]:
#save the cleaned data to the cleaned folder
#df_amazon.write.csv('/mnt/project_mount/cleaned/amazon/', header=True)
df_tv_shows.write.mode('overwrite').csv('/mnt/projectnew_mount/cleaned/netflix/', header=True)

In [0]:
dbutils.fs.unmount('/mnt/projectnew_mount')

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-1137696980438333>:1
----> 1 dbutils.fs.unmount('/mnt/projectnew_mount')

File /databricks/python_shell/dbruntime/dbutils.py:362, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    360 exc.__context__ = None
    361 exc.__cause__ = None
--> 362 raise exc

ExecutionError: An error occurred while calling o1711.unmount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory not mounted: /mnt/projectnew_mount; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory not mounted: /mnt/projectnew_mount
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:135)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:69)
	at com.databricks.backend.daem